In [1]:
%matplotlib qt5

import datetime

import matplotlib.pyplot as mplt
import matplotlib.gridspec as mplgs
import matplotlib.cm as mcmap

from siriuspy.clientarch import OrbitBPM, OrbitSOFB, ClientArchiver
import siriuspy.envars as envars
import numpy as np

In [2]:
clt = ClientArchiver('https://10.0.38.42')

In [3]:
# Create PVArch device (SI Vertical Orbit)
orbitx = OrbitBPM(OrbitBPM.DEVICES.SI_X, connector=clt)
orbity = OrbitBPM(OrbitBPM.DEVICES.SI_Y, connector=clt)

In [4]:
# define start, stop timestamps and averagering window for first retrieval
orbitx.timestamp_start = orbity.Time(2020, 10, 14, 14, 0, 0)
orbitx.timestamp_stop = orbity.Time(2020, 10, 14, 14, 10, 0)
orbity.timestamp_start = orbity.Time(2020, 10, 14, 14, 0, 0)
orbity.timestamp_stop = orbity.Time(2020, 10, 14, 14, 10, 0)

# retrieve first orbit data at 15h
orbitx.update(interpolate=False)
orbity.update(interpolate=False)

In [15]:
orbx, orby = [], []
timex, timey = [], []
namesx = sorted(orbitx._pvdata)
namesy = sorted(orbity._pvdata)

for namex, namey in zip(namesx, namesy):
    pvdatax = orbitx._pvdata[namex]
    pvdatay = orbity._pvdata[namey]
    valuex = np.array(pvdatax.value)
    valuex -= valuex.mean()
    valuey = np.array(pvdatay.value)
    valuey -= valuey.mean()
    timex.append(pvdatax.timestamp)
    timey.append(pvdatay.timestamp)
    orbx.append(valuex)
    orby.append(valuey)
# orbx = np.array(orbx)
# orby = np.array(orby)

In [39]:
f = mplt.figure(figsize=(9,6))
gs = mplgs.GridSpec(2, 1)
gs.update(left=0.12, right=0.8, top=0.95, bottom=0.12)

ax1 = f.add_subplot(gs[0, 0])
ax2 = f.add_subplot(gs[1, 0], sharex=ax1)

indcs = np.arange(43, 54)
timestamps = orbitx._pvdata['SI-01M2:DI-BPM:PosX-Mon'].timestamp
colors = mcmap.jet(np.linspace(0, 1, len(indcs)))

# line = ax1.pcolormesh(orbx, cmap='jet', vmin=0, vmax=orbx.max())
linex = ax1.plot(orbx[indcs])
liney = ax2.plot(orby[indcs])
for i, idx in enumerate(indcs):
    linex[i].set_label(str(datetime.datetime.fromtimestamp(timestamps[idx]))[10:])
    linex[i].set_color(colors[i])
    liney[i].set_color(colors[i])
# for i, (cor, namex, namey) in enumerate(zip(colors, namesx, namesy)):
#     ax2.plot(pvdatay.timestamp, valuey, color=cor)
#     if not i % 8:
#         line.set_label(namex.device_name)
ax1.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax2.set_xlabel('BPM Index')
ax1.set_ylabel('Horizontal [um]')
ax2.set_ylabel('Vertical [um]')
ax1.grid()
ax2.grid()
f.show()

In [33]:
str()

'2020-10-14 10:59:59.500000'

In [ ]:
l.set_label